### Import packages needed

In [1]:
import sys,os
notebook_dir = os.getcwd()  
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.append(parent_dir)
from implementation.Cloth import Cloth 
from implementation.utils import createRectangularMesh
import time
import numpy as np

### Create or load an initial mesh

In [2]:
na = 25; nb = 25
X, T = createRectangularMesh(a = 1,b = 1, na = na, nb = nb, h = 0.125)
X[:,2] += .5; #adjust height

### Call the Cloth class

In [3]:
clothilde = Cloth(X, T); #clothilde.plotMesh()

In [4]:
clothilde.estimateTimeStep(L=1)

Based on your mesh, your best dt is: 0.003139653894152179


np.float64(0.003139653894152179)

### Set simulator parameters:

In [5]:
# solver parameters
dt = 0.003 #time step
tol = 0.005 # up to 0.5% of relative error in constraint satisfaction to stop iterations

#physical parameters
rho = 0.1 #cloth density
delta = 0.1 # aerodynamics parameter: between 0 and rho
kappa = 0.3*1e-4 # stifness or bending resistance
alpha = 0.2 #damping of oscillations
shr = 0.1*1e-4 #allowed shearing resistance
str = 0.005*1e-4 #allowed stretching resistance
mu_f = 0.25 #friction with the floor
mu_s = 0.25 #friction with the cloth itself
thck = 1 #size of the balls

clothilde.setSimulatorParameters(dt=dt,tol=tol,
                                rho=rho,delta=delta,kappa=kappa,shr=shr,
                                str=str,alpha=alpha,mu_f=mu_f,mu_s=mu_s,
                                thck=thck)

### Let's simulate!

In [6]:
clothilde.restart()

In [7]:
inds_ctr = [0,na-1]
start_time = time.time()
for i in range(1000):
    clothilde.simulate(u = clothilde.positions[inds_ctr], control = inds_ctr)
print('Time:',time.time()-start_time)
print('Average iterations',clothilde.total_iters/(len(clothilde.history_pos)-1))

Time: 1.120985507965088
Average iterations 1.199


In [8]:
clothilde.makeMovie(4,True,2)

### Let's stretch up things a little bit

In [9]:
tf = 500; t = np.linspace(0,2*np.pi,tf); 

In [10]:
inds_ctr = [0,na-1]
u = clothilde.positions[inds_ctr]; 
for i in range(tf):
    u[0] -= np.array([0.0015*np.sin(t[i]),0,0])
    clothilde.simulate(u = u, control = inds_ctr)

In [11]:
clothilde.makeMovie(4,True,2)

### And up again

In [12]:
tf = 200; t = np.linspace(0,np.pi,tf); 

In [13]:
inds_ctr = [0,na-1]; freq = 3
u = clothilde.positions[inds_ctr]; 
for i in range(tf):
    u[:,2] += 0.005 #*np.sin(freq*t[i])
    clothilde.simulate(u = u, control = inds_ctr)

In [18]:
inds_ctr = []; 
u = clothilde.positions[inds_ctr]; 
for i in range(400):
    clothilde.simulate(u = u, control = inds_ctr)

In [19]:
clothilde.makeMovie(4,True,2)

### And a final surprise

In [20]:
na = 21; nb = 22
X, T = createRectangularMesh(a = 1,b = 1.4, na = na, nb = nb, h = 0.4)
X[:,2] += 0.9; #adjust height
Y = X.copy(); 
Y[:,1] += 0.75
Y[:,2] -= 0.25
Z = Y.copy()
Z[:,0] += 0.75
Z[:,2] -= 0.15
X = np.concatenate([X,Y,Z]); T = np.concatenate([T,T+na*nb,T+2*na*nb])

In [21]:
clothilde = Cloth(X, T); 
clothilde.setSimulatorParameters(dt=0.003,tol=tol,
                                rho=rho,delta=delta,kappa=kappa,shr=shr,
                                str=str,alpha=alpha,mu_f=mu_f,mu_s=mu_s,
                                thck=thck)
clothilde.plotMesh()

In [22]:
inds_ctr = [na-1,na*nb +na-1, na*nb + na*nb-1]
u = clothilde.positions[inds_ctr]; 
for i in range(1500):
    clothilde.simulate(u = u, control = inds_ctr)

In [23]:
clothilde.makeMovie(4,True,2)

In [24]:
print('Average iterations',clothilde.total_iters/(len(clothilde.history_pos)-1))

Average iterations 1.706
